# NN
Bu not defteri, elektrikli araçların enerji tüketimini tahmin etmek için oluşturulmuş NN modeli oluşturma sürecini detaylandırmaktadır.


### 1. Kütüphaneleri Yükleme

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
import joblib
import random

In [ ]:
# Seed ayarı (tekrar üretilebilirlik için)
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

### 2. Veri Yükleme ve Ön İnceleme


In [ ]:
df = pd.read_csv('final_training_data.csv')
df.head(3)

,timestamp,vehicle_id,speed_kmh,lat,lon,z,acceleration,mass_kg,soc_pc,energy_consumption,...,device.battery.capacity,frontSurfaceArea,recuperationEfficiency,maximumPower,device.battery.maximumChargeRate,rollDragCoefficient,rotatingMass,radialDragCoefficient,dist_m,slope_pct
0,2,veh1,0.000,39.751093,30.502114,825.839847,0.00,1916.0,91.741386,0.00,...,93684.0,2.45,0.93,187368.0,138421.0,0.01,38.0,0.445,NaN,NaN
1,3,veh1,13.824,39.751061,30.502096,825.898326,3.84,1916.0,91.736124,4.93,...,93684.0,2.45,0.93,187368.0,138421.0,0.01,38.0,0.445,3.842938,1.521728
2,4,veh1,27.648,39.750998,30.502060,826.015284,3.84,1916.0,91.721126,14.04,...,93684.0,2.45,0.93,187368.0,138421.0,0.01,38.0,0.445,7.685875,1.521728


In [ ]:
print("Orijinal veri boyutu:", df.shape)
print("Araç sayısı:", df['vehicle_id'].nunique())
print("Eksik veri durumu:")
print(df.isnull().sum())

Orijinal veri boyutu: (481411, 25)
Araç sayısı: 300
Eksik veri durumu:
timestamp                                0
vehicle_id                               0
speed_kmh                                0
lat                                      0
lon                                      0
z                                        0
acceleration                             0
mass_kg                                  0
soc_pc                                   0
energy_consumption                       0
accel                                    0
decel                                    0
propulsionEfficiency                     0
airDragCoefficient                       0
constantPowerIntake                      0
device.battery.capacity                  0
frontSurfaceArea                         0
recuperationEfficiency                   0
maximumPower                             0
device.battery.maximumChargeRate         0
rollDragCoefficient                      0
rotatingMass              

In [ ]:
# 'dist_m' sütunundaki NaN değerleri 0 ile doldurur
df['dist_m'] = df['dist_m'].fillna(0)

# 'slope_pct' sütunundaki NaN değerleri 0 ile doldurur
df['slope_pct'] = df['slope_pct'].fillna(0)

In [ ]:
print(df.isnull().sum())

timestamp                           0
vehicle_id                          0
speed_kmh                           0
lat                                 0
lon                                 0
z                                   0
acceleration                        0
mass_kg                             0
soc_pc                              0
energy_consumption                  0
accel                               0
decel                               0
propulsionEfficiency                0
airDragCoefficient                  0
constantPowerIntake                 0
device.battery.capacity             0
frontSurfaceArea                    0
recuperationEfficiency              0
maximumPower                        0
device.battery.maximumChargeRate    0
rollDragCoefficient                 0
rotatingMass                        0
radialDragCoefficient               0
dist_m                              0
slope_pct                           0
dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481411 entries, 0 to 481410
Data columns (total 25 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   timestamp                         481411 non-null  int64  
 1   vehicle_id                        481411 non-null  object 
 2   speed_kmh                         481411 non-null  float64
 3   lat                               481411 non-null  float64
 4   lon                               481411 non-null  float64
 5   z                                 481411 non-null  float64
 6   acceleration                      481411 non-null  float64
 7   mass_kg                           481411 non-null  float64
 8   soc_pc                            481411 non-null  float64
 9   energy_consumption                481411 non-null  float64
 10  accel                             481411 non-null  float64
 11  decel                             481411 non-null  f

### 3. Özellik Mühendisliği (Feature Engineering)


* **İvme Ayrımı:** İvme değeri (acceleration) pozitif ve negatif olmak üzere iki ayrı sütuna (acc_pos, acc_neg) ayrılır. Bu, pozitif ivmenin (hızlanma) enerji tüketimini artırdığı, negatif ivmenin (yavaşlama/fren) ise rejeneratif enerji kazanımı sağlayabileceği bilgisini modele daha açık bir şekilde iletmemizi sağlar.

In [ ]:
df["acc_pos"] = df["acceleration"].clip(lower=0)
df["acc_neg"] = (-df["acceleration"]).clip(lower=0)

* **Hız Birimi Dönüşümü:** Hız (speed_kmh) birimi km/h'den m/s'ye (speed_ms) dönüştürülür. Fiziksel hesaplamalar için daha uygun bir birimdir.

* **Aerodinamik Katsayı (CdA):** airDragCoefficient ve frontSurfaceArea sütunları birleştirilerek CdA adında yeni bir özellik oluşturulur. Bu iki değerin çarpımı, aerodinamik direncin doğrudan bir göstergesidir.

In [ ]:
# Hız: m/s
df["speed_ms"] = df["speed_kmh"] / 3.6

# Aerodinamik: Cd * A
df["CdA"] = df["airDragCoefficient"] * df["frontSurfaceArea"]


### 4. Gereksiz Sütunları Kaldırma


In [ ]:
columns_to_drop = [
    'timestamp',  # Zaman bilgisi
    'lat', 'lon', 'z',  # GPS koordinatları
    'accel','decel', 'acceleration',
    'speed_kmh',
    'device.battery.capacity',
    'airDragCoefficient', 'frontSurfaceArea',

]
df = df.drop(columns=columns_to_drop)
df.shape

(481411, 18)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481411 entries, 0 to 481410
Data columns (total 18 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   vehicle_id                        481411 non-null  object 
 1   mass_kg                           481411 non-null  float64
 2   soc_pc                            481411 non-null  float64
 3   energy_consumption                481411 non-null  float64
 4   propulsionEfficiency              481411 non-null  float64
 5   constantPowerIntake               481411 non-null  float64
 6   recuperationEfficiency            481411 non-null  float64
 7   maximumPower                      481411 non-null  float64
 8   device.battery.maximumChargeRate  481411 non-null  float64
 9   rollDragCoefficient               481411 non-null  float64
 10  rotatingMass                      481411 non-null  float64
 11  radialDragCoefficient             481411 non-null  f

### 5. Veriyi Eğitim, Doğrulama ve Test Setlerine Ayırma


Aynı araca ait verilerin farklı setlere karışmasını engellemek için GroupShuffleSplit kullanılarak veriler eğitim, doğrulama ve test setlerine ayrılır. Bu sayede veri sızıntısının (data leakage) önüne geçilmiş olur.

In [ ]:

groups = df["vehicle_id"]

X = df.drop(columns=[ "vehicle_id", "energy_consumption"])
y = df["energy_consumption"]


def group_split(X, y, groups, test_size=0.15, val_size=0.15, rs=42):
    """
    X, y ve groups'u alır; önce TEST setini, sonra kalan içinden VALIDATION setini ayırır.
    NOT: Aynı 'vehicle_id'ye ait TÜM satırlar ya train/val tarafında kalır ya da testte olur;
         asla bir araca ait satırlar birden fazla parçaya bölünmez.
    """
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=rs)
    i_tr, i_te = next(gss.split(X, y, groups))
    gss2 = GroupShuffleSplit(n_splits=1, test_size=val_size/(1-test_size), random_state=rs)
    j_tr, j_val = next(gss2.split(X.iloc[i_tr], y.iloc[i_tr], groups.iloc[i_tr]))
    tr_idx = X.iloc[i_tr].index[j_tr]; val_idx = X.iloc[i_tr].index[j_val]; te_idx = X.index[i_te]
    return tr_idx, val_idx, te_idx

train_idx, val_idx, test_idx = group_split(X, y, groups)
X_train, y_train = X.loc[train_idx], y.loc[train_idx]
X_val,   y_val   = X.loc[val_idx],   y.loc[val_idx]
X_test,  y_test  = X.loc[test_idx],  y.loc[test_idx]
print(f"Split -> train:{len(train_idx)}, val:{len(val_idx)}, test:{len(test_idx)}")
print("\nEğitim için kullanılacak özelliklerin bilgi özeti:")
X_train.info()

Split -> train:342307, val:72919, test:66185

Eğitim için kullanılacak özelliklerin bilgi özeti:
<class 'pandas.core.frame.DataFrame'>
Index: 342307 entries, 0 to 481410
Data columns (total 16 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   mass_kg                           342307 non-null  float64
 1   soc_pc                            342307 non-null  float64
 2   propulsionEfficiency              342307 non-null  float64
 3   constantPowerIntake               342307 non-null  float64
 4   recuperationEfficiency            342307 non-null  float64
 5   maximumPower                      342307 non-null  float64
 6   device.battery.maximumChargeRate  342307 non-null  float64
 7   rollDragCoefficient               342307 non-null  float64
 8   rotatingMass                      342307 non-null  float64
 9   radialDragCoefficient             342307 non-null  float64
 10  dist_m                  

StandardScaler veri normalizasyonu yapan bir sınıftır. Her bir sütundaki verinin ortalamasını 0, standart sapmasını 1 olacak şekilde dönüştürür.
Bu sayede tüm özellikler aynı ölçekten geldiği için modelin her feature’a eşit önem vermesi sağlanır.

Biz fit işlemini sadece eğitim verisine yapıyoruz, çünkü validation veya test verisine uygularsak onların ortalama ve standart sapmasını öğrenmiş oluruz.
Bu da modelin eğitim sırasında aslında görmemesi gereken bilgiyi kullanması anlamına gelir. İşte bu duruma data leakage (veri sızıntısı) deniyor. O yüzden:

Eğitim setinde fit + transform yapıyoruz.

Validation ve test setlerinde sadece transform yapıyoruz.

In [ ]:
#Normalizasyon
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
#Dataset Sınıfı
class EnergyDataset(Dataset):
    # Veriyi tensör haline getirip DataLoader ile batch’ler halinde beslememize olanak sağlar.
    def __init__(self, X, y):
        self.X = torch.tensor(X.values if hasattr(X, 'values') else X, dtype=torch.float32)
        self.y = torch.tensor(y.values if hasattr(y, 'values') else y, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_loader = DataLoader(EnergyDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(EnergyDataset(X_val, y_val), batch_size=32, shuffle=False)
test_loader = DataLoader(EnergyDataset(X_test, y_test), batch_size=32, shuffle=False)

Giriş boyutundan başlayıp giderek küçülen (256 → 128 → 64) yapıya sahiptir.

Enerji tüketimi tahmini için giriş verilerimiz normalleştirilmiş olduğu ve negatif değerler gelebileceği için, ReLU bu verilerden pozitif bölgede hızlı öğrenme sağlar. Ayrıca modelin eğitimi sırasında optimizasyonu kolaylaştırır.

Katmanlar arasında BatchNorm kullanarak eğitimi hızlandırmayı ve daha stabil öğrenme sağlamayı hedefledik. Özellikle Dropout ile birlikte kullanıldığında, modelin genelleme kabiliyetini artırır.

Dropout eğitim sırasında, her iterasyonda belirli oranda nöronun bağlantılarını rastgele kapatan bir regularizasyon tekniğidir ve enerji tüketimi tahmininde, modelin eğitim setindeki gürültüye veya belirli örneklere aşırı uyum sağlamasını engelliyor.

In [ ]:
#MLP modeli
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.15),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(X_train.shape[1]).to(device)

MSELoss (Mean Squared Error) regresyon problemleri için yaygın kullanılan kayıp fonksiyonudur. Tahmin ile gerçek değer arasındaki farkın karesini alır, ortalamasını hesaplar.

Adam optimizer, momentum ve adaptif öğrenme oranı özelliklerini birleştirir.

ReduceLROnPlateau: Doğrulama (validation) kaybı belirli bir süre (patience) boyunca iyileşmezse learning rate’i azaltır.

In [ ]:
#Loss, optimizer, schedular
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                 mode='min',  # validation kaybı minimuma inmeye çalışılır.
                                                 factor=0.1,  # learning rate, iyileşme olmazsa mevcut değerinin %10’una düşürülür.
                                                 patience=5,  # 5 epoch boyunca iyileşme olmazsa düşürme işlemi yapılır.
                                                 verbose=True)

In [ ]:
#Eğitim döngüsü
best_val = float('inf')
patience, wait = 10, 0

for epoch in range(25):
    # Train
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    # Validation
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            preds = model(xb.to(device))
            y_true.extend(yb.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    val_mae = mean_absolute_error(y_true, y_pred)
    val_rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    val_r2 = r2_score(y_true, y_pred)
    scheduler.step(val_mae)

    print(f"Epoch {epoch+1} | Val MAE: {val_mae:.4f} | RMSE: {val_rmse:.4f} | R²: {val_r2:.4f}")

    # Early stopping
    if val_mae < best_val:
        best_val = val_mae
        torch.save(model.state_dict(), "best_mlp.pt")
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping.")
            break

Epoch 1 | Val MAE: 1.3362 | RMSE: 2.7021 | R²: 0.9477
Epoch 2 | Val MAE: 1.1979 | RMSE: 2.5842 | R²: 0.9521
Epoch 3 | Val MAE: 1.1366 | RMSE: 2.5319 | R²: 0.9541
Epoch 4 | Val MAE: 0.9784 | RMSE: 2.4476 | R²: 0.9571
Epoch 5 | Val MAE: 1.4135 | RMSE: 2.8037 | R²: 0.9437
Epoch 6 | Val MAE: 1.1821 | RMSE: 2.5787 | R²: 0.9523
Epoch 7 | Val MAE: 1.2446 | RMSE: 2.6105 | R²: 0.9512
Epoch 8 | Val MAE: 1.1546 | RMSE: 2.5458 | R²: 0.9536
Epoch 9 | Val MAE: 1.2589 | RMSE: 2.6176 | R²: 0.9509
Epoch 10 | Val MAE: 1.0559 | RMSE: 2.4856 | R²: 0.9557
Epoch 11 | Val MAE: 1.0646 | RMSE: 2.4882 | R²: 0.9556
Epoch 12 | Val MAE: 1.3374 | RMSE: 2.7419 | R²: 0.9461
Epoch 13 | Val MAE: 1.2142 | RMSE: 2.6070 | R²: 0.9513
Epoch 14 | Val MAE: 1.2078 | RMSE: 2.5903 | R²: 0.9519
Early stopping.


### 7. Model Değerlendirme

Eğitilen model, doğrulama ve test setleri üzerinde tahminler yaparak performansı değerlendirilir. Mean Squared Error (MSE), Root Mean Squared Error (RMSE) ve R-kare (R²) metrikleri kullanılarak modelin başarısı ölçülür.

In [ ]:
# ===== 9. Test değerlendirme =====
model.load_state_dict(torch.load("best_mlp.pt"))
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        preds = model(xb.to(device))
        y_true.extend(yb.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

test_mae = mean_absolute_error(y_true, y_pred)
test_rmse = math.sqrt(mean_squared_error(y_true, y_pred))
test_r2 = r2_score(y_true, y_pred)
print(f"Epoch {epoch+1} | TEST MAE: {test_mae:.4f} | RMSE: {test_rmse:.4f} | R²: {test_r2:.4f}")


Epoch 14 | Val MAE: 1.0478 | RMSE: 2.4693 | R²: 0.9657


===========================================

VALID set:

MAE: 1.20

RMSE: 2.59

R²: 0.95

=============================================

TEST set:

MAE: 1.04

RMSE: 2.46

R²: 0.96

===========================================

Test setinde performans daha iyi
-> MAE ve RMSE testte daha düşük, R² daha yüksek → Model test verisinde validation’dan daha iyi çalışmış.

Genelleme kabiliyeti yüksek
-> Validation ve test skorları birbirine yakın, aşırı bir fark yok

